In [3]:
#pip install  snowflake-connector-python
import snowflake.connector
import pandas as pd
snow_con = snowflake.connector.connect(user='student',password='HSUnivSFTests970',
    account='gkb48589',warehouse='COMPUTE_s',database='DWH',ocsp_fail_open=False)
snow_curs = snow_con.cursor()

print(snow_curs.execute('select count(*) from SF_SAMPLE_DATA.TPCH_SF1.CUSTOMER').fetchone())

(150000,)


In [6]:
create_table_query = """
CREATE TABLE IF NOT EXISTS SF_SAMPLE.PUBLIC.CUSTOMER (
    CUSTOMERID INT AUTOINCREMENT,
    NAME STRING NOT NULL,
    TYPE STRING,
    BALANCE FLOAT
);
"""
snow_curs.execute(create_table_query)

In [8]:
import snowflake.connector
snow_con = snowflake.connector.connect(user='student',password='HSUnivSFTests970',
    account='GKB48589',warehouse='COMPUTE_S',database='SF_SAMPLE',ocsp_fail_open=False)
snow_curs = snow_con.cursor()
print('ku')


import csv
# Specify the CSV file path
csv_file_name = 'customer_data.csv'
csv_file_path = '/tmp/'+'customer_data.csv'
# Get data
sql = 'select * from "SF_SAMPLE"."PUBLIC"."CUSTOMER"'
snow_curs.execute(sql)
f_curs = snow_curs.fetchall()

# Write the data to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for r in f_curs:
        csvwriter.writerow(r)
print(f"Data extracted to {csv_file_path}")

ku
Data extracted to /tmp/customer_data.csv


In [9]:
# loading data


snow_curs.execute("REMOVE @~").fetchall()
#snow_curs.execute("LIST @~").fetchall()
#PUT file command
snow_curs.execute("USE SCHEMA SF_SAMPLE.PUBLIC")
sf_stage = '@~'
put_command = f"""PUT file://{csv_file_path} {sf_stage}"""
print(put_command)
snow_curs.execute(put_command).fetchall()


#Loading command
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
#df = pd.read_csv(csv_file_path)
#snow_curs.execute('USE SCHEMA SF_SAMPLE.PUBLIC')
table_name = "SF_SAMPLE.PUBLIC.CUSTOMER"
copy_into_cmd = f"""
COPY INTO {table_name}
FROM {sf_stage}/{csv_file_name}.gz
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' );
"""
snow_curs.execute(copy_into_cmd).fetchall()

PUT file:///tmp/customer_data.csv @~


[('customer_data.csv.gz', 'LOADED', 72, 72, 1, 0, None, None, None, None)]

In [10]:
snow_curs.execute("USE SCHEMA SF_SAMPLE.PUBLIC")
sf_stage = '@~'
put_command = f"""PUT file://{csv_file_path} {sf_stage}"""
print(put_command)
snow_curs.execute(put_command).fetchall()


PUT file:///tmp/customer_data.csv @~


[('customer_data.csv',
  'customer_data.csv.gz',
  1536,
  0,
  'NONE',
  'GZIP',
  'SKIPPED',
  '')]

In [11]:
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
#df = pd.read_csv(csv_file_path)
#snow_curs.execute('USE SCHEMA SF_SAMPLE.PUBLIC')
table_name = "SF_SAMPLE.PUBLIC.CUSTOMER"
copy_into_cmd = f"""
COPY INTO {table_name}
FROM {sf_stage}/{csv_file_name}.gz
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1);
"""
snow_curs.execute(copy_into_cmd).fetchall()

[('customer_data.csv.gz', 'LOADED', 71, 71, 1, 0, None, None, None, None)]